In [0]:
%sql
-- Creating new database for multiple layers because Unity Catalog is not supported in CE
CREATE DATABASE IF NOT EXISTS edl_hc_datamart.bronze;
CREATE DATABASE IF NOT EXISTS edl_hc_datamart.silver;
CREATE DATABASE IF NOT EXISTS edl_hc_datamart.gold;



In [0]:
%sql
-- CREATE TABLE IF NOT EXISTS edl_hc_datamart.bronze.attendance (
--     date                   DATE,
--     employee_id            INT,
--     present                BOOLEAN,
--     _meta_pipeline_name    STRING,
--     _meta_source_name      STRING,
--     _meta_batch_id         STRING,
--     _meta_run_id           STRING,
--     _meta_schema_version   STRING,
--     _meta_producer_system  STRING,
--     _meta_ingestion_user   STRING,
--     _meta_ingestion_ts     TIMESTAMP,
--     _meta_source_file      STRING,
--     _meta_load_time        TIMESTAMP,
--     _meta_load_date        STRING
-- )
-- USING DELTA;

In [0]:
%sql
-- CREATE TABLE IF NOT EXISTS edl_hc_datamart.bronze.departments (
--     dept_code              STRING,
--     dept_name              STRING,
--     _meta_pipeline_name    STRING,
--     _meta_source_name      STRING,
--     _meta_batch_id         STRING,
--     _meta_run_id           STRING,
--     _meta_schema_version   STRING,
--     _meta_producer_system  STRING,
--     _meta_ingestion_user   STRING,
--     _meta_ingestion_ts     TIMESTAMP,
--     _meta_source_file      STRING,
--     _meta_load_time        TIMESTAMP,
--     _meta_load_date        STRING
-- )
-- USING DELTA;

In [0]:
%sql
-- CREATE TABLE IF NOT EXISTS edl_hc_datamart.bronze.employees (
--     employee_id            INT,
--     first_name             STRING,
--     last_name              STRING,
--     date_of_birth          DATE,
--     dept_code              STRING,
--     email                  STRING,
--     _meta_pipeline_name    STRING,
--     _meta_source_name      STRING,
--     _meta_batch_id         STRING,
--     _meta_run_id           STRING,
--     _meta_schema_version   STRING,
--     _meta_producer_system  STRING,
--     _meta_ingestion_user   STRING,
--     _meta_ingestion_ts     TIMESTAMP,
--     _meta_source_file      STRING,
--     _meta_load_time        TIMESTAMP,
--     _meta_load_date        STRING
-- )
-- USING DELTA;

In [0]:
%sql
-- DROP TABLE IF EXISTS edl_hc_datamart.bronze.jobs;
-- CREATE TABLE IF NOT EXISTS edl_hc_datamart.bronze.jobs (
--     employee_id             BIGINT,
--     end_date                STRING,
--     job_title               STRING,
--     position_id             STRING,
--     start_date              STRING,
--     status                  STRING,

--     salary_amount           DOUBLE,
--     salary_currency         STRING,
--     salary_frequency        STRING,
--     salary_effective_from   DATE,
--     salary_effective_to     DATE,

--     _meta_pipeline_name     STRING,
--     _meta_source_name       STRING,
--     _meta_batch_id          STRING,
--     _meta_run_id            STRING,
--     _meta_schema_version    STRING,
--     _meta_producer_system   STRING,
--     _meta_ingestion_user    STRING,
--     _meta_ingestion_ts      TIMESTAMP,
--     _meta_source_file       STRING,
--     _meta_load_time         TIMESTAMP,   -- added
--     _meta_load_date         STRING       -- added
-- )
-- USING DELTA;

In [0]:
%sql

CREATE TABLE IF NOT EXISTS  edl_hc_datamart.silver.employees (
     employee_id INT,
     first_name STRING,
     last_name STRING,
     email STRING,
     dept_code STRING,
     date_of_birth DATE,
     date_of_birth_fmt STRING,
     is_active BOOLEAN,
     effective_date TIMESTAMP,
     end_date TIMESTAMP
   )
   USING DELTA;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS edl_hc_datamart.silver.jobs (
  employee_id INT,
  position_id STRING,
  job_title STRING,
  start_date DATE,
  start_date_fmt STRING,
  end_date DATE,
  end_date_fmt STRING,
  status STRING,
  salary_amount INTEGER,
  salary_currency STRING,
  salary_frequency STRING,
  salary_effective_from DATE,
  salary_effective_to DATE,
  is_active BOOLEAN,
  effective_date TIMESTAMP,
  effective_end_date TIMESTAMP
)
USING DELTA;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS edl_hc_datamart.silver.departments (
  dept_name STRING,
  dept_code STRING,
  is_active BOOLEAN,
  effective_date TIMESTAMP,
  end_date TIMESTAMP
)
USING DELTA;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS edl_hc_datamart.silver.attendance (
  employee_id INT,
  attendance_date DATE,
  present BOOLEAN,
  attendance_date_fmt STRING,
  is_active BOOLEAN,
  effective_date TIMESTAMP,
  end_date TIMESTAMP
)
USING DELTA;

In [0]:
%sql

CREATE TABLE IF NOT EXISTS  edl_hc_datamart.gold.dim_employees (
     employee_key INT,
     first_name STRING,
     last_name STRING,
     email STRING,
     dept_code STRING,
     date_of_birth_fmt STRING
     )
   USING DELTA;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS edl_hc_datamart.gold.dim_departments (
  dept_name STRING,
  dept_code STRING
)
USING DELTA;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS edl_hc_datamart.gold.dim_date (
  date_key DATE,
  date_label STRING
)
USING DELTA;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS edl_hc_datamart.gold.fact_job_history (
  employee_key INT,
  dept_key STRING,
  start_date_key DATE,
  end_date_key DATE,
  position_id STRING,
  job_title STRING,
  status STRING  
)


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS edl_hc_datamart.audit;

In [0]:
%sql
-- DROP TABLE IF EXISTS edl_hc_datamart.audit.audit_ingestion;

In [0]:
%sql

-- Catalog/Schema is optional; adjust to your environment

CREATE TABLE IF NOT EXISTS edl_hc_datamart.audit.audit_ingestion (
    pipeline_name STRING NOT NULL,
    source_type STRING NOT NULL,
    source_name STRING NOT NULL,
    bronze_table STRING,
    batch_id STRING,
    run_id STRING,
    trigger_type STRING,
    attempt BIGINT,
    run_start_ts TIMESTAMP,
    run_end_ts TIMESTAMP,
    duration_ms BIGINT,
    last_status STRING,
    records_read BIGINT,
    records_written BIGINT,
    error_count BIGINT,
    error_message STRING,
    watermark_col STRING,
    last_success_watermark_value STRING,
    current_run_high_watermark_value STRING,
    file_checkpoint_path STRING,
    schema_version_applied STRING,
    producer_system STRING,
    ingestion_user STRING,
    notes STRING,
    last_load_date DATE,
    created_at TIMESTAMP,
    updated_at TIMESTAMP
)
USING DELTA
PARTITIONED BY (pipeline_name)
TBLPROPERTIES (
  delta.autoOptimize.optimizeWrite = true,
  delta.autoOptimize.autoCompact = true
);